In [ ]:
import pandas as pd
import re

def parse_game(game):
    sections = re.split('\n\s*\n', game.strip())
    game_dict = dict([line.replace("]", '').replace("[", '').split(' ', 1) for line in sections[0].split('\n')])
    game_dict['Moves'] = sections[1]
    return game_dict

with open('ficsgamesdb_202301_blitz2000_movetimes_278527.pgn') as f:
    #next(f)
    games = (parse_game(game) for game in f.read().split('\n\n['))

In [ ]:
df = pd.DataFrame(games)
df.head()

,Event,Site,FICSGamesDBGameNo,White,Black,WhiteElo,BlackElo,WhiteRD,BlackRD,WhiteIsComp,TimeControl,Date,Time,WhiteClock,BlackClock,ECO,PlyCount,Result,Moves,BlackIsComp
0,"""FICS rated blitz game""","""FICS freechess.org""","""530203389""","""konozrout""","""Geforce""","""2060""","""1957""","""27.4""","""15.9""","""Yes""","""300+0""","""2023.01.31""","""23:27:00""","""0:05:00.000""","""0:05:00.000""","""B00""","""55""","""1/2-1/2""",1. e4 {[%emt 0.0]} e5 {[%emt 0.0]} 2. d4 {[%em...,NaN
1,"""FICS rated blitz game""","""FICS freechess.org""","""530203352""","""Geforce""","""pikozrout""","""1951""","""2137""","""15.9""","""39.5""",NaN,"""300+0""","""2023.01.31""","""23:17:00""","""0:05:00.000""","""0:05:00.000""","""B40""","""69""","""1/2-1/2""",1. e4 {[%emt 0.0]} c5 {[%emt 0.0]} 2. Nf3 {[%e...,"""Yes"""
2,"""FICS rated blitz game""","""FICS freechess.org""","""530203324""","""Geforce""","""konozrout""","""1957""","""2054""","""15.9""","""27.5""",NaN,"""300+0""","""2023.01.31""","""23:03:00""","""0:05:00.000""","""0:05:00.000""","""B52""","""134""","""0-1""",1. e4 {[%emt 0.0]} c5 {[%emt 0.0]} 2. Nf3 {[%e...,"""Yes"""
3,"""FICS rated blitz game""","""FICS freechess.org""","""530203266""","""GimmeDatKing""","""Frubes""","""2292""","""1796""","""74.1""","""49.4""",NaN,"""300+0""","""2023.01.31""","""22:43:00""","""0:05:00.000""","""0:05:00.000""","""B90""","""55""","""1-0""",1. e4 {[%emt 0.0]} c5 {[%emt 0.0]} 2. Nf3 {[%e...,NaN
4,"""FICS rated blitz game""","""FICS freechess.org""","""530203175""","""playoften""","""GimmeDatKing""","""1826""","""2290""","""24.8""","""74.5""",NaN,"""300+0""","""2023.01.31""","""22:22:00""","""0:05:00.000""","""0:05:00.000""","""A04""","""42""","""0-1""",1. Nf3 {[%emt 0.0]} f5 {[%emt 0.0]} 2. d4 {[%e...,NaN


In [ ]:
# Cleaning

def clean_df(df): # Depricated
    datetime_columns = ['Date', 'Time', 'WhiteClock', 'BlackClock']
    float_columns = ['WhiteRD', 'BlackRD']
    int_columns = ['FICSGamesDBGameNo', 'WhiteElo', 'BlackElo', 'PlyCount']
    
    for column in df.columns:
        
        if df[column].dtype != object or not isinstance(df.iloc[0][column], str): # Keeps from cleaning the data twice
            continue
        if column in float_columns:
            df[column] = df[column].apply(lambda f: float(f.replace('"', '')))
        elif column in int_columns:
            df[column] = df[column].apply(lambda n: int(n.replace('"', '')))
        elif column in datetime_columns:
            df[column] = df[column].apply(lambda d: pd.to_datetime(d.replace('"', '')))
        else:
            df[column] = df[column].apply(lambda s: str(s).replace('"', '')).astype('category')
    
    df['WhiteIsComp'] = df['WhiteIsComp'] == 'Yes'
    df['BlackIsComp'] = df['BlackIsComp'] == 'Yes'
    
    return df

In [ ]:
cleaned = clean_df(df)
print(cleaned.shape)
print(cleaned.dtypes)
cleaned.head()

(2461, 20)
Event                      category
Site                       category
FICSGamesDBGameNo             int64
White                      category
Black                      category
WhiteElo                      int64
BlackElo                      int64
WhiteRD                     float64
BlackRD                     float64
WhiteIsComp                    bool
TimeControl                category
Date                 datetime64[ns]
Time                 datetime64[ns]
WhiteClock           datetime64[ns]
BlackClock           datetime64[ns]
ECO                        category
PlyCount                      int64
Result                     category
Moves                      category
BlackIsComp                    bool
dtype: object


,Event,Site,FICSGamesDBGameNo,White,Black,WhiteElo,BlackElo,WhiteRD,BlackRD,WhiteIsComp,TimeControl,Date,Time,WhiteClock,BlackClock,ECO,PlyCount,Result,Moves,BlackIsComp
0,FICS rated blitz game,FICS freechess.org,530203389,konozrout,Geforce,2060,1957,27.4,15.9,True,300+0,2023-01-31,2023-03-14 23:27:00,2023-03-14 00:05:00,2023-03-14 00:05:00,B00,55,1/2-1/2,1. e4 {[%emt 0.0]} e5 {[%emt 0.0]} 2. d4 {[%em...,False
1,FICS rated blitz game,FICS freechess.org,530203352,Geforce,pikozrout,1951,2137,15.9,39.5,False,300+0,2023-01-31,2023-03-14 23:17:00,2023-03-14 00:05:00,2023-03-14 00:05:00,B40,69,1/2-1/2,1. e4 {[%emt 0.0]} c5 {[%emt 0.0]} 2. Nf3 {[%e...,False
2,FICS rated blitz game,FICS freechess.org,530203324,Geforce,konozrout,1957,2054,15.9,27.5,False,300+0,2023-01-31,2023-03-14 23:03:00,2023-03-14 00:05:00,2023-03-14 00:05:00,B52,134,0-1,1. e4 {[%emt 0.0]} c5 {[%emt 0.0]} 2. Nf3 {[%e...,False
3,FICS rated blitz game,FICS freechess.org,530203266,GimmeDatKing,Frubes,2292,1796,74.1,49.4,False,300+0,2023-01-31,2023-03-14 22:43:00,2023-03-14 00:05:00,2023-03-14 00:05:00,B90,55,1-0,1. e4 {[%emt 0.0]} c5 {[%emt 0.0]} 2. Nf3 {[%e...,False
4,FICS rated blitz game,FICS freechess.org,530203175,playoften,GimmeDatKing,1826,2290,24.8,74.5,False,300+0,2023-01-31,2023-03-14 22:22:00,2023-03-14 00:05:00,2023-03-14 00:05:00,A04,42,0-1,1. Nf3 {[%emt 0.0]} f5 {[%emt 0.0]} 2. d4 {[%e...,False


#### Random Forest

Our second model to test is Random Forest. Let's do a quick grid search on a few of our most important yet relatively simple features and test our prediction accuracy.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RepeatedKFold
from scipy.stats import norm

In [ ]:
X = df[['BlackElo', 'WhiteElo', 'BlackIsComp', 'WhiteIsComp']]
y = df['Result']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:

params = {'n_estimators':[100], 'ccp_alpha': [.005]}
model = RandomForestClassifier()
rkf = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)
gs = GridSearchCV(model, params, cv=rkf)
gs.fit(X_train,y_train)

print('Best params: ', gs.best_params_)

Best params:  {'ccp_alpha': 0.005, 'n_estimators': 100}


In [ ]:
print('Rough test score using our best estimator: ', gs.best_estimator_.score(X_test, y_test))
print('Rough training score using our best estimator: ', gs.best_estimator_.score(X_train, y_train))

Rough test score using our best estimator:  0.702922077922078
Rough training score using our best estimator:  0.7040650406504065


The best parameter in this grid is `n_estimators=500`. At first glance, it seems that random forests are very effective in predicting the outcomes of chess games in this problem. About 66.07% of our  model's predictions are accurate, which is alright. It's likely that our estimators are overfit, so we may need to apply some pruning methods. We will continue to explore hyperparameters that can further improve these results, and we will play around with creating some features to see if we can improve this already impressive accuracy. We aren't sure exactly which features we will construct, but we have ideas to of creating ones that reveal the play style of the player through certain statistics such as win/loss ratio against the current opponent.